In [ ]:
from IPython.display import clear_output
!pip install pandas pyarrow
clear_output()

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import gc 

In [ ]:
def csv_to_parquet(CSV_PATH,NEW_PARQUET_PATH):
    chunksize = 100

    csv_stream = pd.read_csv(CSV_PATH, sep='\t', chunksize=chunksize, low_memory=True)

    for i, chunk in enumerate(csv_stream):
        print("Chunk", i)
        if i == 0:
            # Guess the schema of the CSV file from the first chunk
            parquet_schema = pa.Table.from_pandas(df=chunk).schema
            # Open a Parquet file for writing
            parquet_writer = pq.ParquetWriter(NEW_PARQUET_PATH, parquet_schema, compression='snappy')
        # Write CSV chunk to the parquet file
        table = pa.Table.from_pandas(chunk, schema=parquet_schema)
        parquet_writer.write_table(table)

    parquet_writer.close()
    
    gc.collect()
    

In [ ]:
%%time

TRAIN_CSV_PATH = "../input/ubiquant-market-prediction/train.csv"
NEW_TRAIN_PARQUET_PATH ="train.parquet"

csv_to_parquet(TRAIN_CSV_PATH,NEW_TRAIN_PARQUET_PATH)